In [27]:
import os, json
import pandas as pd
import glob
from geopy.geocoders import Nominatim
import numpy as np
from tqdm import tqdm
import multiprocessing as mp
from multiprocesspandas import applyparallel
tqdm.pandas()

In [29]:
path_to_json = 'C:\\Users\\'
geolocator = Nominatim(user_agent='my_user_agent')

temp = pd.DataFrame()
json_pattern = os.path.join(path_to_json, '*.json')
file_list = glob.glob(json_pattern)

for file in file_list:
    data = pd.read_json(file, orient = str)
    temp = temp.append(data, ignore_index = True)

temp2 = pd.json_normalize(temp['list'])

BIC_DATA = temp.join(temp2)
del temp
del temp2
del file_list

BIC_DATA = BIC_DATA.drop('id',1)
BIC_DATA = BIC_DATA.drop('list',1)
BIC_DATA.insert(0, "id", range(1, 1+len(BIC_DATA)))
BIC_DATA['geocode'] =  BIC_DATA['city']+','+BIC_DATA['country']


In [3]:
BIC_DATA

,id,country,country_code,bank,city,branch,swift_code,geocode
0,1,Andorra,AD,ANDORRA BANC AGRICOL REIG S.A.,LES ESCALDES,None,BACAADAD,"LES ESCALDES,Andorra"
1,2,Andorra,AD,ANDORRA GESTIO AGRICOL REIG SAU,ESCALDES-ENGORDANY,None,AAMAADAD,"ESCALDES-ENGORDANY,Andorra"
2,3,Andorra,AD,BANC SABADELL D'ANDORRA S.A.,ANDORRA LA VELLA,None,BSANADAD,"ANDORRA LA VELLA,Andorra"
3,4,Andorra,AD,BANCA PRIVADA D'ANDORRA S.A.,LES ESCALDES,None,CASBADAD,"LES ESCALDES,Andorra"
4,5,Andorra,AD,"CREDIT ANDORRA,S.A.",ANDORRA LA VELLA,None,CRDAADAD,"ANDORRA LA VELLA,Andorra"
...,...,...,...,...,...,...,...,...
112882,112883,Zimbabwe,ZW,THE TETRAD GROUP,HARARE,None,TEGRZWH1,"HARARE,Zimbabwe"
112883,112884,Zimbabwe,ZW,TRUST CORPORATE SECURITIES (PRIVATE) LIMITED,HARARE,None,TCSPZWH1,"HARARE,Zimbabwe"
112884,112885,Zimbabwe,ZW,ZIMBABWE BANKING CORPORATION LTD.,BULAWAYO,None,ZBCOZWHXBYO,"BULAWAYO,Zimbabwe"
112885,112886,Zimbabwe,ZW,ZIMBABWE BANKING CORPORATION LTD.,HARARE,None,ZBCOZWHX,"HARARE,Zimbabwe"


In [77]:
LOC_DATA = pd.DataFrame(BIC_DATA['geocode'])
LOC_DATA = LOC_DATA.drop_duplicates()
LOC_DATA = LOC_DATA[LOC_DATA.geocode != 'CENTRO DE CIUDAD SANTA FE,Mexico']
LOC_DATA = LOC_DATA[LOC_DATA.geocode != 'MONTERREY,N.L.,Mexico']
location = LOC_DATA.geocode.progress_apply(lambda loc: geolocator.geocode(loc))

100%|██████████| 16634/16634 [2:18:43<00:00,  2.00it/s]  


In [78]:
location.to_csv('location.csv')


In [76]:
tmp = LOC_DATA.copy()
tmp["latitude"] = [x.latitude for x in location]
tmp["longitude"] = [x.longitude for x in location]

AttributeError: 'NoneType' object has no attribute 'latitude'

In [79]:
location

0         (les Escaldes, Escaldes-Engordany, AD700, Ando...
1         (Escaldes-Engordany, AD700, Andorra, (42.48691...
2         (Andorra la Vella, AD500, Andorra, (42.5069391...
9         (دبي, 39205, الإمارات العربية المتحدة, (25.265...
12        (نادي العاصمة للفروسية +971501222551, أبو ظبي,...
                                ...                        
112823    (Harare, Harare Province, 04-263, Zimbabwe, (-...
112858    (Bulawayo, Bulawayo Province, Zimbabwe, (-20.1...
112870    (Chinhoyi, Makonde, Mashonaland West, Zimbabwe...
112871    (Gweru, Midlands, Zimbabwe, (-19.4616316, 29.8...
112873    (Mutare, Manicaland, Zimbabwe, (-19.24886155, ...
Name: geocode, Length: 16634, dtype: object

In [ ]:
BIC_DATA["latitude"] = [x.latitude for x in location]
BIC_DATA["longitude"] = [x.longitude for x in location]
BIC_DATA = BIC_DATA.drop('geocode',1)

In [48]:
for i in split_dataframes:
    temp = pd.DataFrame(split_dataframes[i])
    location = temp.geocode.progress_apply(lambda loc: geolocator.geocode(loc))
    LOC_DATA["latitude"] = [x.latitude for x in location]
    LOC_DATA["longitude"] = [x.longitude for x in location]
    LOC_DATA.to_csv('split_'+str(i)+'.csv')
    

TypeError: list indices must be integers or slices, not DataFrame

In [80]:

LOC_DATA["latitude"] = [x.latitude for x in location]

AttributeError: 'NoneType' object has no attribute 'latitude'